In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Followed this repo
https://github.com/krishnaik06/TFOD/blob/main/object_detection_camera.ipynb

In [1]:
# pip install tensorflow

In [2]:
import tensorflow as tf
print(tf.__version__)

2.18.0


Create the data directory
The snippet shown below will create the data directory where all our data will be stored. The code will create a directory structure as shown bellow:

.. code-block:: bash

data
└── models
where the models folder will will contain the downloaded models.

In [4]:
import os
os.getcwd()

'c:\\Users\\spurusho\\Downloads\\ObjectDetection\\ObjectDetection\\models\\research\\object_detection'

In [22]:
import os
DATA_DIR = os.path.join(os.getcwd(), 'data')
MODELS_DIR = os.path.join(DATA_DIR, 'models')
for dir in [DATA_DIR, MODELS_DIR]:
    if not os.path.exists(dir):
        os.mkdir(dir)

# Download the model
The code snippet shown below is used to download the object detection model checkpoint file, as well as the labels file (.pbtxt) which contains a list of strings used to add the correct label to each detection (e.g. person).

The particular detection algorithm we will use is the SSD ResNet101 V1 FPN 640x640. More models can be found in the TensorFlow 2 Detection Model Zoo <https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md>_. To use a different model you will need the URL name of the specific model. This can be done as follows:

Right click on the Model name of the model you would like to use;
Click on Copy link address to copy the download link of the model;
Paste the link in a text editor of your choice. You should observe a link similar to [download.tensorflow.org/models/object_detection/tf2/YYYYYYYY/XXXXXXXXX.tar.gz](http://);
Copy the XXXXXXXXX part of the link and use it to replace the value of the MODEL_NAME variable in the code shown below;
Copy the YYYYYYYY part of the link and use it to replace the value of the MODEL_DATE variable in the code shown below.
For example, the download link for the model used below is: [download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz](http://)

In [25]:
import tarfile
import urllib.request

# Download and extract model
MODEL_DATE = '20200711'
MODEL_NAME = 'ssd_mobilenet_v2_320x320_coco17_tpu-8'
MODEL_TAR_FILENAME = MODEL_NAME + '.tar.gz'
MODELS_DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/'
MODEL_DOWNLOAD_LINK = MODELS_DOWNLOAD_BASE + MODEL_DATE + '/' + MODEL_TAR_FILENAME
PATH_TO_MODEL_TAR = os.path.join(MODELS_DIR, MODEL_TAR_FILENAME)
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))
if not os.path.exists(PATH_TO_CKPT):
    print('Downloading model. This may take a while... ', end='')
    urllib.request.urlretrieve(MODEL_DOWNLOAD_LINK, PATH_TO_MODEL_TAR)
    tar_file = tarfile.open(PATH_TO_MODEL_TAR)
    tar_file.extractall(MODELS_DIR)
    tar_file.close()
    os.remove(PATH_TO_MODEL_TAR)
    print('Done')

# Download labels file
LABEL_FILENAME = 'mscoco_label_map.pbtxt'
LABELS_DOWNLOAD_BASE = \
    'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/'
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))
if not os.path.exists(PATH_TO_LABELS):
    print('Downloading label file... ', end='')
    urllib.request.urlretrieve(LABELS_DOWNLOAD_BASE + LABEL_FILENAME, PATH_TO_LABELS)
    print('Done')

C:\Users\spurusho\AppData\Local\Temp\ipykernel_26632\698677122.py:17: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar_file.extractall(MODELS_DIR)


Done


In [5]:
# # This Colab requires a recent numpy version.
# !pip install numpy==1.24.3
# !pip install protobuf==3.20.3

In [33]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.3-cp312-cp312-win_amd64.whl.metadata (168 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl (8.0 MB)
Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl (220 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.55.3-cp312-cp312-win_amd64.whl (2.2 MB)
Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl (71 kB)
Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl (2.6 MB)
Using cached pyparsing-3.2.1-py3-none-any.


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   ---------------------------------------- 0.2/43.6 MB 6.9 MB/s eta 0:00:07
    --------------------------------------- 0.7/43.6 MB 8.8 MB/s eta 0:00:05
    --------------------------------------- 0.7/43.6 MB 5.7 MB/s eta 0:00:08
   - -------------------------------------- 1.2/43.6 MB 6.8 MB/s eta 0:00:07
   - -------------------------------------- 1.5/43.6 MB 7.0 MB/s eta 0:00:07
   - -------------------------------------- 1.9/43.6 MB 7.0 MB/s eta 0:00:06
   -- ------------------------------------- 2.3/43.6 MB 7.4 MB/s eta 0:00:06
   -- ------------------------------------- 2.6/43.6 MB 7.3 MB/s eta 0:00:06
   -- ------------------------------------- 2.9/43.6 MB 7.4 MB/s eta 0:00:06
   -- -


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# pip install tensorflow==2.17.0

In [40]:
pip install tensorflow_hub

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.7 MB 1.9 MB/s eta 0:00:01
   ----- ---------------------------------- 0.3/1.7 MB 2.6 MB/s eta 0:00:01
   -------------- ------------------------- 0.6/1.7 MB 4.2 MB/s eta 0:00:01
   ------------------------ --------------- 1.0/1.7 MB 5.5 MB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.7 MB 5.7 MB/s eta 0:00:01
   ---------------------------------- ----- 1.5/1.7 MB 5.5 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 5.2 MB/s eta 0:00:01
   ------------------------------------ --- 1.6/1.7 MB 5.2 MB/s eta 0:00:01
   -------------


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import pathlib

import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

C:\Users\spurusho\AppData\Local\Temp\ipykernel_27756\3955447678.py:7: DeprecationWarning: scipy.misc is deprecated and will be removed in 2.0.0
  import scipy.misc


In [10]:
model = hub.load('C:/Users/spurusho/Downloads/ObjectDetection/ObjectDetection/models/research/object_detection/data/models/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model')

In [12]:
import urllib
urllib.request.urlretrieve('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxITEhUSEhMWFRUVFxUSFhgVFxYVFhcVFRgWFhcVFRUYHSggGBolGxYVITEhJSkrLi4uFx8zODMtNygtLisBCgoKDg0OGBAQGi0lHyUrLS0tLSs3Ky8tLS0tLS0tLS0tLS0rLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLS0tLf/AABEIANAA8wMBIgACEQEDEQH/xAAcAAEAAgIDAQAAAAAAAAAAAAAABgcBBQIECAP/xABVEAACAQIDAgcKCgQKCAcAAAABAgADEQQSIQUxBgcTIkFRkxQXIzJSVGFxgZEkM1NykqGxwdHSFUJz0zRDYnSisrPC4fEIFjVEY5SjtFWCg4TE4vD/xAAaAQEBAQEBAQEAAAAAAAAAAAAAAQIDBAYF/8QAKhEBAQABAwIFAwQDAAAAAAAAAAERAgNSEzESFCFBoQSB8DIzUVMicbH/2gAMAwEAAhEDEQA/ALxiIgIiICIiAiIMBEj+0+GuAoMyVK92U2YU0q1ip6m5JWsdRoeua8cZezflKv8Ay2JP1CneMrhMImv2LtvD4pC+HqBwDZtGVlPU6MAyn0ECbCEa7bW3MNhED4mslJToM5sWI1sq72PoE0B40NkeeL2db8krv/SCb4RhgfIrAej4k/fIbwbpDkjcA887wD0LJlcL176OyPPB2db8kHjR2R52Oyr/ALuVCKK+SPcIyDqHuEp6Ld76WyPO/wDpV/yTHfT2T51/0q/5JUeQdQ90yFHUIPRbZ40tk+cnsq35JlONHZJNu6Tr0mlWA9pySpLX9c0PCAc9fm/eZKSSvU+HrrURXRgyOAyspBVlIuCCN4In0kV4rCf0Vhb+Qw91R5KpUIifJcQhOUMpPUCL+6B9YiICIiAiIgIiICJgmAYGYiICIiAkJ4ztr1aGFrNSNitJmB6ibi8mrSueOH+CVx/wT98lWIbsrYlLk0ZcLVctdrsLaljvfK4Zusnp6Tvm0wGwVQk1MM7g+UwIHqFKjf6pH9sbTr01wy06rovc1M2ViovnqAnT1Ca39NYo/wC8Vu0f8Z5tWv1pd3HomvB1Ew22KTUFNNMQ5o1E1Cle52cG1hrytFtSARzhYaiXFKX2fVY19l1GJLGtQDEm5JajXBuek8+XOZ6Z2hblRn+kGPD4b1VvsoSIcF/ijfyz9iyf8eGCWrVoakMM4BuLXYUgAw3gE2FxuJEgHBoeCPzz9iyTuvs2xiYmRNMsgTBiZGunTAwBNLwhFnXry/f9skbVMoA0va27dfX2/wCcje3/AB1P8k/bJezWnuv3isFtk4S/yZPvdz9822P2utMMehQWPqAuZq+LVLbKww/4ZPvZjNbw1Ypg8Uw3ihXPuRjCNVTrVMXhRiq5Nd6tMVqeFNQ08OquAVplVFqhy72cNre2UG06qO2QKdl4MdS50sOrQUDaRnGbdrUWFFMgWmlNBdSTYU09M7dPa1UeOUJ0B0RFBYXCC98zWI9Gsz4tXtG/SLE2NijTq01Rm5Oo2Rqbszqt1YhqRfnLzlUZb5bE6AyYyquB+Pq1KyhyCq1qJXm5SCRXUhtd4ykS1ZuXLFIiIQiJxaBm8TGWIGSIAmYgIiICYJmYgYAld8cA+C1/2LfY0sWV9xuj4LX/AGFT+q0lWILX2FUxNPD1KbIAlEU2zcpfMKlRgBlQg6Ou7rnyTgVXJAz0xchRda41O4XNIC83/BPaFOmiZmy2A3tbQMt8qnRjzT6dN2ovvsDiVvY1i5NSm4LF0FlYk6ubDQjdbxROHgluV1bcty0NXAvh3wCPYsmKwq6bjzFXT3y4DKw4Uv4TBvvHdOHINyb5XpLvOu++v3S0J6J2iVXnGhwXxOJNOthUFR0zAqWCc1sjG19Cc1JN5Gl5Wuz+B216ale4HNyWuXpDU2/leiejYkwZefhwX2v5ge0pfmmRwV2x5ge0pfnnoCJTLz+vBXa/mB7Sl+edr/VXaqrcYIlj1VKXN3/y7G4tL2iDKg14JbXP+4n21aP7ycKnF1tSs658OKY3XNSkQB16OTL/AIkMtbwf2Z3NhaWHvfk0CX67SJ8YhtgMX+wqj3qR98n0r3jGQnAYsddNh7yBFIq7hG3wmr863uAH3QdooRzg1yLGwUgnLlJBOqkrYHQzjwjN8TWt8o3t1mszTE7NXusDi7x3KVtRbw+HPo3Ylj9v1S6JR/FbTPLg9HL0R7qWKb7peE3GaRESo4kzIEzEBERAREQEREBERASCca9LNhao66NQfUZO5EOMeiWw7AdKVB9UlWd1fcGGJp021saRYeCFQa5jv6Brv6JJtngck9rbzfwVukXzk6P6h65E+BGHq9y0X5PP4MKOTq5Da2hYEgHfa2u4g9cl2ApV8pU0KpF9PCUyB6FBYZT6Zy93WvhwtNjhCb6YmhfMoU25XD71Gg06ui0s+VRwqzcthqTKqjl8GyqGzNz63OzG+/wIOl7a69Vrzt7OVIiIQiQjhrtnEnEUNn4SpyNStmZ6uUMadOmMzZQf1jew9R3aEdGrwIqWLNtXaZ0JOWv1DoVU+oQLFiVm3AwggfpPapvr8dUO/dup2/CdteAZ/wDE9pf8z/8AWFWDEr/ZVXE4DHUMLUxFXE4bFCoENchqtKrSGYg1N7KwIH4W51gQhINxm83AYpupL/0lk5kc4c7RWjhajm2gtzrWudADfT3wsV/U2Ng6rNVKklyWJDMRc+pwPqmDwcwXyZ/pfvJHNnNUpKRh6VaslQmq70u5kpmq2jhRVoO9tB0gejpPaXG43owWJ9tXCL/8WWbWn+L8r46l/BanRTE4ejQQL4Zqr6kk5cPXUEjWwu4F79IlnSqeLPbBOLNKpTq06r8pmSoFa2VV5y1URVIIQaWA1W17m9rSYk7JSIiEIiICIiAmM0EwBAzERAREQE6m0sAtZMjew9U7MyIFOY7gDtHD1G7jy1KLaqvKmkUNyTlBBFiTu6J104O7fOgplP8A3VO39nLsiTEa8VVjwS4vcUMWuN2jVVmp2NOmrvVsR4pLMAFAJJsoNz0gXBs6IlZYJgGY3zlAqLjMx1Shj+WpNlqJg8SyNYGzBAQbEEH2iVsOMna3njdlh/3csPjgHwk+nB4we6kplIRFqW98navnjenweH+rwc4njJ2t543Z0P3cik3eyqIFMPa7MamoUMwFNb5aYbTOb3vY6AwidcA+EGKxmJwT4mqarJiqqKSqLZTQQ25gHTL9lDcAKd62FbUMuLbeEDaUwjBinNY6XBA3D0S+YCQvjF+KAvaz0j6+eukmkhvGMp5B/Rkb2Kyk/UDCx99m005FSyr0gH+VmNgBY6jLewOvUbztYtUBF6Chc1ibroxFhmsDYb9/TaaygCadMlfEblAbgjmu+W62BFyQDruE72IwoFNgUZczqWuVbUvcAWtfnHeej13i0jW4eiE2lRK5QrLuW3jGniLjQ6aU1MnEhddbY/BaEatvN/4nFfjJpLUIiJAiYMxA5REQMATMRAREQEREDFpmIgdbaOJ5Omz9X36TzttTjP2u9apyeJ5NA7KqrTo2AViBqyEk6dc9AcIT4BvZ9s8t0KSk1CRfwlQdPXMarfZ12/DJbqb7vg7Z6McTu/iqP30+iS3i/wCG20amLo0sRXFenVORgUpKUuujKUUG+bSx6/Rc16KYHRr7dP8AGSPgB/tDD/PX+ug/vTnnXn1dZq2tWZJ64eioiJ3eVT3HEPhA9OEx31UVlHS8+OMfCF/mm0PqoJKNAiLTLOxhcayAqMrKSCVcZluNzW6D6RPjmtoN3T6ZxI90IsTi0xbvXwxNtMYoAUWUDkxoAPW3vno2ea+Kv47D/wA9T66f+E9KQEivD9fg1X9m1vomSqR3hnRzUWHWpHvBhY6+FI5E85hzanQ7L4zi5Goy7ujrn1KgAkA0wGXnMhFtQDlyre9tOjWdPYtZTQUlwuZARnyhTmQELcruOY353SZ3nqpoBUu11Nne6j06sQfcdZajoYgg43B6secbFr6jksR0Ekj6pMpEMc47vwfOJOa2oy3PI4omwsNNN9pL4CIiQJi0zEBERAREQEREBERAREQNXwk/g7+yeX8HpynXyr+zX7Z6h4R/wd/Z9s8t4NgDUBI+Mf7ZjV3bkzortSQcAf4fQ+en9rSkeDjrHvEkXARgMbQ/aUhf/wBWnMW9l2NN8V/1XoyIidnNUHHIPhCfzTaP/bpKOJ6B/nLz44z4dP5rtH/tf8JRULSck+rpmFEMfdCJ5xY/HYe27u2j9dN9/p0npWeZuLJ7VaH8/wAKPelQT0zATq4/BiopU9M7UQK+wmyto4C6YZFr0bmwBAcDoDIzKGI3Zg66AXBOs7qbY2m2i4BlPW3JW/tx98mkS5EV2PwerNXXF4xlNRMxpomoUsrJdnsL2ViAoUAZiSWJuJVESBERAREQEREDF5m84mBA5REQEREDBMwI3zlA1vCL+Dv7PtnlEYcvWZF3mpUHuZiT7gT7J6x28t6D+q/uInlvZ+2+4cfy2TMadZ2Kn9ZSWuB7DvnPXMu+zcZbHYPBw4l2p0M5qICxJy5bDfmQC6LqNczegG9psuBVMrj6KMCGWtTVl6QVrIGGnUQfdOni+H7d0PXo1XpI5LclTAUC5JNrDU85tWv43VYDnxfbZ5faiZhlatUJBBvZmcVDb2Bpzuj1ldtG5/jqlemLzM4zlPQ8SquM7AtXx2GoKQGrUsbRUtfKGfDZQTYE21kLHExjvlsN9Kr+7lucOeDFbE8lXwlRaeJoNylPP4jEAgo3UCDY+qdHD4nbZHOwOGB3G+KIv6QAjWHtvC1WT8TuNH8fQsNTblSAOs8zWfXvKYzznDj2VD/dlm1G2ydO5MLY77Yp9fR8T9k+qVdsdOEwg9WJqE/2P3wiA7G4FPs6rhOVrK5qY6gRkBAuodbc75/1S7hIPsvg/jsRi6WK2hyVNMOWahQokvd2FuUqud5G8W9G7XNOYCV1tjhpXGIakpyKMoXk6a1KjZsxuTUdVUAKSTr6AbyxZR3CTEGlizUUAkWFmuRuPUR1zOq4b04mbUyp4vEE3bE4gjU81sOD6lXkQPewnfwRqvry2LTqzth9fVbN9krpeF2IGgWl9FvzTcjhViCMyLTII0OQkDm+Kxzc2zXvfoMzNTPUTepjKtJWfl2fIMxWqlM3A6MyKlvXrbqMkgMqL/W2tUfkQKRWotRWIVr2s1rHNvsAZbdLxR6hNy5W3Mc4iJWSYYwTMAQGsTlEBERAREQEwRMxAREQOrtT4mp8xvsnmDa1Om2IxGdQSGIU8qKdrIWN1tzvF9Gth0ieoNpfFVPmN9k8ybUoscVilXN8YG0Sk48RzryhBGhO49G4m057j2/RWeO5mflrVwFPJmut+cbcqmigGwK2zXup9lpJOB1Gmm1MEKaqvhchtVFW5C1ATYeKCQN80xzcgp8JbK+uWjl3uBZ75uncRr7BJBwZDfpPBZsw+EEjMtFehxvpHXf1ADcJy05y/U39OibWrEna+z0XE41HCgk6AamQ9+NDZQLL3T4pKnmPvU2NrjXXpnpfPJlEhh40tlecf0HP3QONLZXnFvXTqD+7C4qZmYAkNPGjsrzn3I/4THfU2V5wezqfhBhNIkK76myvOD9Bx9omz2Pw1weJJFFnYKUVmNOoFBqGyc4i1idL7oMJFKL4ar8Ib2f3h90vSUhwzQnFFRvawHRu5UnX1CZ1dlnao2ROM7aYJidGQ3/lrb2m+k5YXZrVATTekwBsSKi75yxXLFc9g/Hp/wCf+o09BpuHqlJbG4N1wxq8wrTV3chwbDI3QOmXam4eqdNExGp2coiJtWLTMRAREQEREBERAREQEREDDC4sdx0kM2txYbNxFQ1alNgzb8rWB94MmkQsuEAXif2WP1Kn0h+WbPYXF3s7CVVrUqJ5RfFLMTY9dt0lkSYh4r/Lq7T+Kf1Tyk6p+k7Mtxy1NQLc39Xf9c9W7U+Kf5pnlSsh7vuQQO6KRBtv6N/3wRPzRpfJU/oL+E4NTpH+Lp/QX8Jh2nC8YH1WjS+Sp/QX8J2Eo0/k0+gv4TrJPqjwr47co0+56xFNARSqHRVB0U6ggaRxVYkeH0tm7lCgbh8Jdram/wCqZx223wat+yf+qZ8uLhiGrF75r4UsD0eFrG/ukndfZ6BlTbVoI+NrCogcLSNQZiVClWcZi4IKrYkE+ncRLZlT7doX2iVuwPJM4ylr3WowuUUHlBztVII6baaWrt92cNilK1kFqnLouHY8qGshz2C0yisGJqPzGZicp3WsflS2JRV6jZiVcNTCiqWNns/McrzULJlQWt404PT56pmDktYGpTDAkkMFzjKRfLrUI3hLWOabXE5lDEnRnIuStiaiFOeAekkBtwyjSxuYVzosEplULMqpUUEsSAnJ5iCBlUjrUrewW3jCWLT3D1CViQbVWJJAUk3JKqBTF2FQLl01Oe9yRaxyCWbR8VfUPslTVPSOcREOZERAREQE4ATO+coCIiAiIgYvAmLTIEDMREBETBMDqbV+Kf5pnlPamPJx6r4qrXQ9SgXW89W7THgX+aZ5RxeGR3cvcsBpusSzHeSdBe+4HU9EYXKbHGUvlE+mv4zAxlL5RPpr+MiH6GoG+VXJIvTzcmt16KjktzF1TTpz79NeQ2NhzqEqc5cy5uTXmc052u3NuDopte66mBL+7KXyifTX8Z28G1Nhc1VsTlFmUkkC5tr0DpkH/QdDXwdQABXN8nMp6FmcZhziM5Cm17LvvNhhMJhhTZHVkC5jmOVgtKoOcr5WBDko9hp8XAkXCQouGrEOD4NtCVvqDa1iQQdd01PFy9jVFwdcMLqbgjuioND0+NPk+EwzkUAlXJmGfKE5lNnLaLm33C6G25uuavg9SRMZTy3t3RQUC5GnKJa4BsddZZpzVtxHqyRjhTwHw2NZajNUpVUvlqUmAbXeDmBFr69ck8SJp1XTcxWz8XeOX4rapK9VagtQ7rasW13dUzS4C7SB/wBoUU9KYVL+q9xLHJmAJMO3mdz+f+Idg+AC5lbE4qvXytnykhKZN72KC5y+i9pM4iVy1a9WrvSIiGSYvBmLQOUREBERAREQEREBERAREQExaZiB1dpjwT/NM8mV6iqWzlRcg6+OCGYEAbhre4a3i6T17OnU2XQYlmo0iTqSUQkn0kiB5RTF0rLd01Y5roDzQBlckHXUnmrbxdb6CcamLSykZWJLEjIotflApJuBcAqQq83Rb7iD6tOxcL5vR7NPwnH9A4Tzah2VP8IXLy4mOpDLmCnfuphjdixzMGZQbXAC3y80aHUnj3XTIA5q2Vr2pg3foLEsecdfQvRvnqX9AYTzWh2VP8Jj9AYTzWh2VP8ACUy8v4LGU1dSRTcBqR8JSupFmz5grAkAkXG9rCxGt/vsiqDjaWgBbE0DZRYC9RDlUdHUJ6YPB3B+aYfsaf5Zyw2wMJTYPTwtBHXUMtKmrD1MBcSy4S+sbGIiZGCJmIgIiICIiAiIgIiIFNd8LH+WnZrHfCx/lp2azRYTHqlNk5NWLZucbbiBl0Km+VlDDW2p03Edl9q0rkjDIL6gc0gEKqjTJqBYmx0JY3vpb9y7OjhPh85PqNzH7l+W1Th/tAkAOhJIAHJrqToBOdfh5tFGKuyBhvHJr65pTtZAWK0FXMDuyaG1PKU5nNAKE2HlmxBAI+jbaplsxw1M84trYmxUjKxK87nHNrfq0sCJ0dPCfC9fX/Zflsu+Fj/LTs1jvhY/y07NZrMNtWigUHCq2XyihLak848nc7x0/qi1hdT0toYtajBlprTsoUhAACbk5rKAAdQPZLNnRn9E+Ev1G5j03L8pB3wsf5admsd8LH+WnZrIrE30NrjGPM73KpV3wsf5admsd8LH+WnZrIrEdDa4w8zvcqlXfCx/lp2ax3wsf5admsisR0NrjDzO9yqVd8LH+WnZrHfCx/lp2ayKxHQ2uMPM73KpV3wsf5admsd8LH+WnZrIrEdDa4w8zvcqlXfCx/lp2ax3wsf5admsisR0NrjDzO9yqVd8LH+WnZrHfCx/lp2ayKxHQ2uMPM73KpV3wsf5admsd8LH+WnZrIrEdDa4w8zvcqlXfCx/lp2ax3wsf5admsisR0NrjDzO9yqVDjBx/lp2YnIcPtoeUul/4odG+RnB4lqbh0tmW9r6jUEHT1EzuJtusAQCtiSx0O/KFGoPQFFr9Ivv1mbsaPbRG9P1G5767+fdujw+2j5S9XxQ/wD3QfdMd8DaHlJru8GNenT6pqF29WBJGTW4PNvoSxI1O7nHT8Tf5VtsVWIJy3BJ0FrkqFN/YOiZ6OnhPz7LfqNX9l/Pu3nfA2h5admJjvh4/wAtOzWafC7bq0/Fy+LlvbUi5OuvpM419sVHVlYJZlC6KBaxGo6iQLeo2l6OnP6J+fY8xrx+5fz7t13wsf5admsSKxN9Da4xz8zvcq//2Q==','image.jpg')

('image.jpg', <email.message.Message at 0x23d9ee07fe0>)

In [15]:
import cv2

img = cv2.imread('C:/Users/spurusho/Downloads/ObjectDetection/ObjectDetection/models/research/object_detection/image.jpg')
img.shape

(208, 243, 3)

In [16]:
input_img = np.array(img)
input_img.shape

(208, 243, 3)

In [17]:
# Add a new dimension
input_img = np.expand_dims(input_img, axis=0)

In [18]:
input_img.shape

(1, 208, 243, 3)

In [19]:
# running inference
results = model(input_img)

# different object detection models have additional results
# all of them are explained in the documentation
result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

dict_keys(['raw_detection_boxes', 'detection_multiclass_scores', 'detection_classes', 'detection_boxes', 'raw_detection_scores', 'num_detections', 'detection_anchor_indices', 'detection_scores'])


In [20]:
# Clone the tensorflow models repository
# !git clone --depth 1 https://github.com/tensorflow/models

In [22]:
os.getcwd()

'c:\\Users\\spurusho\\Downloads\\ObjectDetection\\ObjectDetection\\models\\research\\object_detection'

In [30]:
# %%bash
# sudo apt install -y protobuf-compiler
# cd models/research/
# protoc object_detection/protos/*.proto --python_out=.
# cp object_detection/packages/tf2/setup.py .
# python -m pip install .


In [24]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

ModuleNotFoundError: No module named 'object_detection'

In [16]:
label_id_offset = 0
image_np_with_detections = input_img.copy()

# Use keypoints if available in detections
keypoints, keypoint_scores = None, None
if 'detection_keypoints' in result:
  keypoints = result['detection_keypoints'][0]
  keypoint_scores = result['detection_keypoint_scores'][0]

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections[0],
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False,
      keypoints=keypoints,
      keypoint_scores=keypoint_scores,
      keypoint_edges=COCO17_HUMAN_POSE_KEYPOINTS)

plt.figure(figsize=(24,32))
plt.imshow(image_np_with_detections[0])
plt.show()

NameError: name 'viz_utils' is not defined